# Identifying Leaf Species using  Convolutional Neural Networks
Based on Kaggle competition: https://www.kaggle.com/c/leaf-classification

For a more detailed look, please see my github: https://github.com/scotttarlow/Leaf_Kaggle

by Scott Tarlow

## Introduction

There are around 1/2 million species of plants in the world. Using images of plants to identify species be useful for a variety of reasons: crop and food supply management, plant based research, species population tracking. In this project I will use Convolutional Neural Networks to classify grey-scale images to identify each image as one of 99 leaf species. These models will either incorporate multiple convolutional layers as well as merging the features extracted by the convolutions with a pre-extracted feature set (provided by Kaggle) of each leaf image. Each model will be evaluated by minimizing the loss function, log-loss. The best of my Neural Networks earned a log-loss of 0.02312 on Kaggle's test data set, which translates to 100% accuracy on predicting a leaf's species from its corresponding image.

## Exploritory Data Analysis and Preprocessing



Below is an example of all the images for the species "Acer Capillipes" In the training data set:

In [1]:
from IPython.display import Image
Image(url='leaf_gifs/Acer_Capillipes.gif')

As you can see, the leaf, depending on the image, varies slightly in size, shape and orientation. We also note that there are 10 images per species in the training set (see distribution below):

In [2]:
Image(url='report_figs/leaf_distro.png')

This note is important because if there was not a uniform distribution of images the training distribution could affect the accuracy of the model. Another observation is that for all the images, the resolution varies. This will make inputting the images into my Convolutional Neural Network (CNN) difficult. To solve this, I will resize the images to 64x64 pixels. The code to do this is given on my github: https://github.com/scotttarlow/Leaf_Kaggle/blob/master/lib/load_images.py

which takes the original image, converts it to a grey scale numpy array, finds the largest axis (by amount of pixels), converts the smaller axis to the same amount of pixels as the larger axis, centers the leaf in the image, and then downscales the image into a 64x64 square image:

Original Image

In [3]:
Image(url='report_figs/leaf_normal.png')

Resized Image to the largest axis (and centered leaf)

In [4]:
Image(url='report_figs/leaf_resize.png')

Image Resized to 64x64

In [5]:
Image(url='report_figs/leaf_interp.png')

I performed various tests with different resolutions, and model accuracy did not improve significantly with images at 96x96, 128x128,256x256 and 512x512 resolutions. However, computation time increased dramatically when increasing image resolution; 64x64 seemed like the best balance of resolution and computation time. 

The numerical pre-extracted features contain 192 features on each leaf's margin, shape and texture. A leaf's margin refers to the structure of a leaf's edge, for example, is the edge smooth or "bumpy"? A leaf's shape refers to the overall profile of the leaf, is the leaf long and slender or ovicular? Finally, a leaf's texture refers to how uniform or heterogeneous the scales on the leaf are. How these features were extracted for the data set are described in this paper (https://github.com/scotttarlow/Leaf_Kaggle/blob/master/PLANT%20LEAF%20CLASSIFICATION%20USING%20PROBABILISTIC%20INTEGRATION%20OF%20SHAPE%2C%20TEXTURE%20AND%20MARGIN%20FEATURES%20-%20Charles%20Mallah.pdf)

The numerical features were preprocessed (https://github.com/scotttarlow/Leaf_Kaggle/blob/master/lib/load_pre_data.py)
using standard scaler.

To split the training set into a training and validation, I used Stratified Shuffle Split, with a Training set of 792 images and a validation set of 192 images. 

Finally, the training labels were converted from string labels such as "Acer Capillipes" to One-Hot Encoded vectors for input into Keras.

## Modeling

All the proposed models are CNNs. A Convolutional layer (the basis for a CNN) is a model in which uses descriptive filters to inform the hierarchical learning process. During the convolutional process, the filters (of smaller size and width than the input image) compute the dot product, helping the model learn to activate only when specific features are captured by the filters. In theory, the more filters an image goes through, the better the network learns, however more convolutions also increase the risk of overlearning. Below is a block diagram of a CNN with 1 convolutional layer:

In [6]:
Image(url='saved_models/1_CNN.h5.png')

The convolutional layer includes 8 filters, at 3 pixels high and 3 pixels wide, a Relu activation layer to account for non-linearities, and a 2x2 max pooling layer. Subsequent models (2 layer and 5 layer) networks repeat this pattern to make a deeper network, however as the model deepens overlearning becomes more rapid. For more detail on this, see the notebooks in this folder on my github (https://github.com/scotttarlow/Leaf_Kaggle/tree/master/model_ipynb).

Deviating from this pattern is the most successful neural network I trained, which includes 2 convolutional layers and a concatenated fully connected layer that merges the features learned from the convolutions with the features Kaggle provides:

In [7]:
Image(url='saved_models/2_CNN_and_pre_connected.h5.png')

One might wonder, why would adding 192 features could greatly affect the performance of a model when the convolutional features number over 600,000? The l1 regularization combined with the drop out player makes it so the number of features that affect the model are much lower than the total number of features in the model. Because the features that Kaggle provides are known to be predictive, these features carry more weight than most of the convolutional features.

## Results

$$\begin{array}{c|c|c} \text{Model} & \text{Minimum Log-Loss} & \text{Maximum Validation Accuracy}  \\ \hline
\text{1_CN} & 1.61 & 0.6364 \\ \hline
\text{2_CN} & 1.42 & 0.6061 \\ \hline
\text{5_CN} & 1.39 & 0.6700 \\ \hline
\text{2_CNN_and_Pre_connected} & 0.02312 & 1.00 \\ \hline
\end{array}$$

The table above provides the minimum validation log-loss and validation accuracy for each model. Each model was run for 100 epochs, with the exception of 2_CNN_and_Pre_connected which was run for 300 epocs. 

Without the numerical data provided by Kaggle, The models were able to achieve a log-loss ranging from 1.39 to 1.61 and a validation accuracy between 0.6061 to 0.6700. One convolutional layer seemed to gain most of the information from the images, however, there was modest improvement by adding another convolution. Going deeper than 2 convolutional layers increased the speed in which the model over fit and did not provide a meaningful gain in log-loss. 


In [8]:
Image(url='report_figs/all_models_validation_loss.png')

A plot of the validation log-loss of all the models confirms the results in the previous table. As you can see, after about 10-20 epochs, the CNNs without the numerical data start to ovefit, while the model with the Kaggle provided nuermical Data never overfits, even at 100 epochs.By far the best model is 2_CNN_and_Pre_connected, which has 2 convolutional layers as well as a fully connected layer that merges the Kaggle data. This model was run for 300 epochs and received a much lower log-loss and 1.00 accuracy.  Because we this model is so successful, we can actually view the first layer of the convolutions:

In [8]:
Image(url='report_figs/Leaf_1_Convolution.png')

The top image in the figure above is the original leaf image, while the bottom 8 images are the convolutional filters in the first convolutional layer. As you can see, the first convolution are detecting area's of great contrast in the images, providing a clear picture of the leaf's edge. Because these images are black and white, the second convolutional layer (not shown) does not provide clear features when viewing the filters. This should be intuitive because the 2nd convolutional layer only marginally improves the log-loss score of the model.

## Conclusions

In this project, 4 CNNs were tested in their ability to correctly label an image of a leaf. The results show that for this image set convolutions learn the edges of the leaf's image to identify which species the leaf is. Furthermore, the Kaggle provided numerical data proved to be very predictive of the leaf species, and when combined with the feature engineering of the CNNs the models were able to label the images with great certainty. Deep Learning often suggests that domain knowledge is as important as a proper neural network structure, however, Kaggle's provided data was extracted using a scientist's expertise in leaves. Not only does this project provide evidence that CNNs cam be used to label images with great precision, but, that domain knowledge can go a long way in increasing a model's accuracy.